<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Literature_match_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import pandas as pd

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [29]:
Auto_5_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_185_TF_rt2.csv', error_bad_lines=False)
Auto_4_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PUF_120k_172_TF_rt2.csv', error_bad_lines=False)
Auto_3_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto3_PUF_120k_165_TF_rt2.csv', error_bad_lines=False)
Auto_2_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto2_PUF_120k_149_TF_rt2.csv', error_bad_lines=False)
Auto_1_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto1_PUF_120k_138_TF_rt2.csv', error_bad_lines=False)
Zafira_1_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Zafira_1_PUF_63_TF_rt2.csv', error_bad_lines=False)
Zafira_2_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Zafira_2_PUF_65_TF_rt2.csv', error_bad_lines=False)


Auto_5_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PF_120k_183_TF_rt2.csv', error_bad_lines=False)
Auto_4_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PF_120k_179_TF_rt2.csv', error_bad_lines=False)
Auto_3_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto3_PF_120k_170_TF_rt2.csv', error_bad_lines=False)
Auto_2_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto2_PF_120k_147_TF_rt2.csv', error_bad_lines=False)
Auto_1_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto1_PF_120k_125_TF_rt2.csv', error_bad_lines=False)
Zafira_1_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Zafira_1_PF_64_TF_rt2.csv', error_bad_lines=False)
Zafira_2_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Zafira_2_PF_66_TF_rt2.csv', error_bad_lines=False)

U181_60k_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/U181_7_TF_rt2.csv', error_bad_lines=False)
U181_120k_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/U181_21_TF_rt2.csv', error_bad_lines=False)
U166_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/U166_26_TF_rt2.csv', error_bad_lines=False)
U164_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/U164_60_TF_rt2.csv', error_bad_lines=False)


QC_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/QC_PUF4_2_120k_159_TF_rt2.csv') #PUF4 120k blank
QC_solvent_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/QC_Solvent_28_5_1_5_120k_184_TF_rt.csv')
QC_perfume_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Perfume_29_8_2023_38_TF_rt.csv')
QC_60k_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/QC_PUF4_2_60k_176_TF_rt2.csv')
QC_pf_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/QC_PF_120k_161_TF_rt2.csv')

<ipython-input-29-60e343315ab9>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Auto_5_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_185_TF_rt2.csv', error_bad_lines=False)
<ipython-input-29-60e343315ab9>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Auto_4_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PUF_120k_172_TF_rt2.csv', error_bad_lines=False)
<ipython-input-29-60e343315ab9>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Auto_3_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto3_PUF_120k_165_TF_rt2.csv', error_bad_lines=False)
<ipython-input-29-60e343315ab9>:4: FutureWarning: The error_bad_lines argument has

In [30]:
Auto_5_gp_df.fillna(0, inplace=True)
Auto_4_gp_df.fillna(0, inplace=True)
Auto_3_gp_df.fillna(0, inplace=True)
Auto_2_gp_df.fillna(0, inplace=True)
Auto_1_gp_df.fillna(0, inplace=True)
Zafira_1_gp_df.fillna(0, inplace=True)
Zafira_2_gp_df.fillna(0, inplace=True)

Auto_5_pm_df.fillna(0, inplace=True)
Auto_4_pm_df.fillna(0, inplace=True)
Auto_3_pm_df.fillna(0, inplace=True)
Auto_2_pm_df.fillna(0, inplace=True)
Auto_1_pm_df.fillna(0, inplace=True)
Zafira_1_pm_df.fillna(0, inplace=True)
Zafira_2_pm_df.fillna(0, inplace=True)

U181_60k_df.fillna(0, inplace=True)
U181_120k_df.fillna(0, inplace=True)
U166_df.fillna(0, inplace=True)
U164_df.fillna(0, inplace=True)

QC_df.fillna(0, inplace=True)
QC_solvent_df.fillna(0, inplace=True)
QC_perfume_df.fillna(0, inplace=True)
QC_60k_df.fillna(0, inplace=True)
QC_pf_df.fillna(0, inplace=True)
# Convert specific columns to float
float_columns = ['SI', 'RHRF Score', 'Total Score']

for df in [Auto_5_gp_df, Auto_4_gp_df, Auto_3_gp_df, Auto_2_gp_df, Auto_1_gp_df, Zafira_1_gp_df, Zafira_2_gp_df, Auto_5_pm_df, Auto_4_pm_df, Auto_3_pm_df, Auto_2_pm_df, Auto_1_pm_df, Zafira_1_pm_df, Zafira_2_pm_df, U181_60k_df, U181_120k_df, U166_df, U164_df, QC_df, QC_solvent_df, QC_perfume_df, QC_60k_df, QC_pf_df]:
    df[float_columns] = df[float_columns].apply(pd.to_numeric, errors='coerce')

In [31]:
Auto_5_gp_df['Ion Type'] = 'gp'
Auto_4_gp_df['Ion Type'] = 'gp'
Auto_3_gp_df['Ion Type'] = 'gp'
Auto_2_gp_df['Ion Type'] = 'gp'
Auto_1_gp_df['Ion Type'] = 'gp'
Zafira_2_gp_df['Ion Type'] = 'gp'
Zafira_1_gp_df['Ion Type'] = 'gp'

Auto_5_pm_df['Ion Type'] = 'pm'
Auto_4_pm_df['Ion Type'] = 'pm'
Auto_3_pm_df['Ion Type'] = 'pm'
Auto_2_pm_df['Ion Type'] = 'pm'
Auto_1_pm_df['Ion Type'] = 'pm'
Zafira_2_pm_df['Ion Type'] = 'pm'
Zafira_1_pm_df['Ion Type'] = 'pm'

Auto_5_gp_df['Sample Name'] = 'Auto_5'
Auto_4_gp_df['Sample Name'] = 'Auto_4'
Auto_3_gp_df['Sample Name'] = 'Auto_3'
Auto_2_gp_df['Sample Name'] = 'Auto_2'
Auto_1_gp_df['Sample Name'] = 'Auto_1'
Zafira_2_gp_df['Sample Name'] = 'Auto_2'
Zafira_1_gp_df['Sample Name'] = 'Auto_1'

Auto_5_pm_df['Sample Name'] = 'Auto_5'
Auto_4_pm_df['Sample Name'] = 'Auto_4'
Auto_3_pm_df['Sample Name'] = 'Auto_3'
Auto_2_pm_df['Sample Name'] = 'Auto_2'
Auto_1_pm_df['Sample Name'] = 'Auto_1'
Zafira_2_pm_df['Sample Name'] = 'Auto_2'
Zafira_1_pm_df['Sample Name'] = 'Auto_1'

In [32]:
print(Auto_3_pm_df.columns)

Index(['Component Name', 'Retention Time', 'Reference m/z', 'Area', 'Height',
       'TIC', 'Formula (mol ion)', 'CAS No.', 'SI', 'RSI', 'HRF Score',
       'RHRF Score', 'Total Score', 'Selected Column Type', 'Calculated RI',
       'Library RI', 'Delta RI', 'Library Name', 'Library ID Number',
       'Sample Name', 'File name', 'Ion Type'],
      dtype='object')


In [33]:
print('Auto5_gp:', Auto_5_gp_df.shape)
print('Auto4_gp:', Auto_4_gp_df.shape)
print('Auto5_pm:', Auto_5_pm_df.shape)
print('Auto4_pm:', Auto_4_pm_df.shape)

Auto5_gp: (749, 22)
Auto4_gp: (851, 22)
Auto5_pm: (321, 22)
Auto4_pm: (321, 22)


In [34]:
def area_ratio(df):
    """
    Calculates ratio of area to the internal standard and saves it in a new column
    """
    benzyl_benzoate_area = df.loc[df['Component Name'] == 'Benzyl Benzoate', 'Area'].values[0]
    df['Area Ratio'] = df['Area'] / benzyl_benzoate_area

    return df

def filter_dataframe(df):
    """
    Removing the results below the probability threshold and disregarding rows with 'Peak' in 'Compound Name'
    """
    condition1 = df['HRF Score'] >= 90
    condition2 = df['SI'] >= 800
    condition3 = df['Delta RI'] <= 5
    condition4 = (df['Total Score'] >= 90) & ~(condition1 & condition2 & condition3)
    condition5 = ~df['Component Name'].str.contains('Peak', case=False)

    filtered_df = df[condition1 & condition2 & condition3 | condition5 | condition4]

    return filtered_df

def rearrange_by_area_ratio(df):
    """
    Rearrearanging the rows in descending fashion from the peak area ratio
    """
    if 'Area Ratio' not in df.columns:
        raise ValueError("DataFrame must have a 'Area Ratio' column")

    df_sorted = df.sort_values(by='Area Ratio', ascending=False)

    #Reset the index to reflect the new order
    df_sorted = df_sorted.reset_index(drop=True)

    return df_sorted

import pandas as pd

def find_common_row(sample_df, ref_df):
    result_df = pd.DataFrame(columns=sample_df.columns)

    for _, sample_row in sample_df.iterrows():
        matching_rows = find_matching_rows(sample_row, ref_df)
        if not matching_rows.empty:
            best_match = find_best_match(sample_row, matching_rows)
            if best_match is not None and sample_row['Area Ratio'] >= (0.001 * best_match['Area Ratio']):
                result_df = result_df.append(sample_row, ignore_index=True)

    result_df = result_df[result_df['Component Name'].isin(ref_df['Component Name'])]

    return result_df

def qc_correction(sample_df, ref_df):
    result_df = sample_df.copy()
    matched_component_names = set()

    for _, sample_row in sample_df.iterrows():
        matching_rows = find_matching_rows(sample_row, ref_df)

        if not matching_rows.empty:
            best_match = find_best_match(sample_row, matching_rows)

            if best_match is not None:
                if sample_row['Area Ratio'] >= (0.001 * best_match['Area Ratio']):
                    matched_component_names.add(best_match['Component Name'])

    result_df = result_df[~result_df['Component Name'].isin(matched_component_names)]

    return result_df

def find_matching_rows(sample_row, ref_df):
    matching_rows = ref_df[ref_df['Formula (mol ion)'] == sample_row['Formula (mol ion)']]
    return matching_rows

def find_best_match(sample_row, matching_rows):
    best_match = None
    min_diff = float('inf')

    for _, ref_row in matching_rows.iterrows():
        diff = compare_strings(sample_row['Component Name'], ref_row['Component Name'])
        if diff < min_diff:
            min_diff = diff
            best_match = ref_row

    return best_match

def compare_strings(str1, str2):
    return 0 if str1 == str2 else 1

In [35]:
Auto_5_gp_df = area_ratio(Auto_5_gp_df)
Auto_4_gp_df = area_ratio(Auto_4_gp_df)
Auto_3_gp_df = area_ratio(Auto_3_gp_df)
Auto_2_gp_df = area_ratio(Auto_2_gp_df)
Auto_1_gp_df = area_ratio(Auto_1_gp_df)
Zafira_2_gp_df = area_ratio(Zafira_2_gp_df)
Zafira_1_gp_df = area_ratio(Zafira_1_gp_df)

Auto_5_pm_df = area_ratio(Auto_5_pm_df)
Auto_4_pm_df = area_ratio(Auto_4_pm_df)
Auto_3_pm_df = area_ratio(Auto_3_pm_df)
Auto_2_pm_df = area_ratio(Auto_2_pm_df)
Auto_1_pm_df = area_ratio(Auto_1_pm_df)
Zafira_2_pm_df = area_ratio(Zafira_2_pm_df)
Zafira_1_pm_df = area_ratio(Zafira_1_pm_df)


U181_60k_df = area_ratio(U181_60k_df)
U181_120k_df = area_ratio(U181_120k_df)
U166_df = area_ratio(U166_df)
U164_df = area_ratio(U164_df)

QC_df = area_ratio(QC_df)
QC_solvent_df = area_ratio(QC_solvent_df)
QC_60k_df = area_ratio(QC_60k_df)
QC_pf_df = area_ratio(QC_pf_df)
print(rearrange_by_area_ratio(QC_df))

                                     Component Name  Retention Time  \
0                           2,4-Di-tert-butylphenol          13.724   
1                                   Benzyl Benzoate          18.606   
2                               1,2-Benzisothiazole          10.349   
3                                            Nonane           4.354   
4                          Butylated Hydroxytoluene          13.673   
..                                              ...             ...   
588                                     Peak@11.905          11.905   
589                                     Peak@16.012          16.012   
590                                     Peak@24.900          24.900   
591                                      Peak@5.196           5.196   
592  L-Leucine, N-cyclobutylcarbonyl-, methyl ester          22.002   

     Reference m/z        Area     Height        TIC Formula (mol ion)  \
0       191.142929  2209640031  388091698  684423124           C14H22O   

In [36]:
Auto_5_gp_df_filtered = qc_correction(Auto_5_gp_df, QC_df)
Auto_4_gp_df_filtered = qc_correction(Auto_4_gp_df, QC_df)
Auto_3_gp_df_filtered = qc_correction(Auto_3_gp_df, QC_df)
Auto_2_gp_df_filtered = qc_correction(Auto_2_gp_df, QC_df)
Auto_1_gp_df_filtered = qc_correction(Auto_1_gp_df, QC_df)
Zafira_2_gp_df_filtered = qc_correction(Zafira_2_gp_df, QC_60k_df)
Zafira_1_gp_df_filtered = qc_correction(Zafira_1_gp_df, QC_60k_df)

Auto_5_pm_df_filtered = qc_correction(Auto_5_pm_df, QC_pf_df)
Auto_4_pm_df_filtered = qc_correction(Auto_4_pm_df, QC_pf_df)
Auto_3_pm_df_filtered = qc_correction(Auto_3_pm_df, QC_pf_df)
Auto_2_pm_df_filtered = qc_correction(Auto_2_pm_df, QC_pf_df)
Auto_1_pm_df_filtered = qc_correction(Auto_1_pm_df, QC_pf_df)
Zafira_2_pm_df_filtered = qc_correction(Zafira_2_pm_df, QC_60k_df)
Zafira_1_pm_df_filtered = qc_correction(Zafira_1_pm_df, QC_60k_df)


U181_60k_df_filtered = qc_correction(U181_60k_df, QC_60k_df)
U181_120k_df_filtered = qc_correction(U181_120k_df, QC_df)
U166_df_filtered = qc_correction(U166_df, QC_60k_df)
U164_df_filtered = qc_correction(U164_df, QC_60k_df)

print('Auto5_gp:', Auto_5_gp_df_filtered.shape)
print('Auto4_gp:', Auto_4_gp_df_filtered.shape)
print('Auto5_pm:', Auto_5_pm_df_filtered.shape)
print('Auto4_pm:', Auto_4_pm_df_filtered.shape)
#Auto_5_gp_df_filtered.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_blank_corrected.csv')

Auto5_gp: (658, 23)
Auto4_gp: (749, 23)
Auto5_pm: (286, 23)
Auto4_pm: (274, 23)


In [37]:
Auto5_gp_comon_df = find_common_row(Auto_5_gp_df, QC_df)
Auto4_gp_comon_df = find_common_row(Auto_4_gp_df, QC_df)
Auto3_gp_comon_df = find_common_row(Auto_3_gp_df, QC_df)
Auto2_gp_comon_df = find_common_row(Auto_2_gp_df, QC_df)
Auto1_gp_comon_df = find_common_row(Auto_1_gp_df, QC_df)


Auto5_pm_comon_df = find_common_row(Auto_5_pm_df, QC_df)
Auto4_pm_comon_df = find_common_row(Auto_4_pm_df, QC_df)
Auto3_pm_comon_df = find_common_row(Auto_3_pm_df, QC_df)
Auto2_pm_comon_df = find_common_row(Auto_2_pm_df, QC_df)
Auto1_pm_comon_df = find_common_row(Auto_1_pm_df, QC_df)

print('Auto5_gp:', Auto5_gp_comon_df.shape)
print('Auto4_gp:', Auto4_gp_comon_df.shape)
print('Auto5_pm:', Auto5_pm_comon_df.shape)
print('Auto4_pm:', Auto4_pm_comon_df.shape)
#print(rearrange_by_area_ratio(Auto5_gp_comon_df))
#Auto5_gp_comon_df.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_blank_matches.csv')

<ipython-input-34-44281c84f980>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)
<ipython-input-34-44281c84f980>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)
<ipython-input-34-44281c84f980>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)
<ipython-input-34-44281c84f980>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)
<ipython-input-34-44281c84f980>:48: FutureWarning: The frame.append method i

Auto5_gp: (91, 23)
Auto4_gp: (102, 23)
Auto5_pm: (48, 23)
Auto4_pm: (49, 23)


In [38]:
Auto5_gp_ordered = rearrange_by_area_ratio(Auto_5_gp_df_filtered)
Auto4_gp_ordered = rearrange_by_area_ratio(Auto_4_gp_df_filtered)
Auto3_gp_ordered = rearrange_by_area_ratio(Auto_3_gp_df_filtered)
Auto2_gp_ordered = rearrange_by_area_ratio(Auto_2_gp_df_filtered)
Auto1_gp_ordered = rearrange_by_area_ratio(Auto_1_gp_df_filtered)


Auto5_pm_ordered = rearrange_by_area_ratio(Auto_5_pm_df_filtered)
Auto4_pm_ordered = rearrange_by_area_ratio(Auto_4_pm_df_filtered)
Auto3_pm_ordered = rearrange_by_area_ratio(Auto_3_pm_df_filtered)
Auto2_pm_ordered = rearrange_by_area_ratio(Auto_2_pm_df_filtered)
Auto1_pm_ordered = rearrange_by_area_ratio(Auto_1_pm_df_filtered)
print(Auto5_gp_ordered.head())
print(Auto4_gp_ordered.head())
print(Auto5_pm_ordered.head())
print(Auto4_pm_ordered.head())

                                      Component Name  Retention Time  \
0                                  1-Butene-3-ethoxy           4.802   
1  Benzenepropanoic acid, 3,5-bis(1,1-dimethyleth...          33.695   
2              Carbonic acid, tetradecyl vinyl ester           9.683   
3                                  Decane, 3-methyl-           6.708   
4                 Carbonic acid, decyl undecyl ester          11.236   

   Reference m/z        Area     Height         TIC Formula (mol ion)  \
0      45.033504  8317593860  886789709  3169610421            C6H12O   
1     219.174301  3487699203  201822395  2147150518          C35H62O3   
2      43.054165  2929291121  479368820  2724804750          C17H32O3   
3      43.054165  1872334548  296600545  1007438567            C11H24   
4      43.054165  1854075664  323340139  2458815240          C22H44O3   

      CAS No.     SI    RSI  ...  Selected Column Type  Calculated RI  \
0   1476-08-0  633.0  648.0  ...      StandardNonPolar 

In [39]:
Auto5_gp_final = filter_dataframe(Auto_5_gp_df_filtered)
Auto4_gp_final = filter_dataframe(Auto_4_gp_df_filtered)
Auto3_gp_final = filter_dataframe(Auto_3_gp_df_filtered)
Auto2_gp_final = filter_dataframe(Auto_2_gp_df_filtered)
Auto1_gp_final = filter_dataframe(Auto_1_gp_df_filtered)
Zafira_2_gp_df_final = filter_dataframe(Zafira_2_gp_df_filtered)
Zafira_1_gp_df_final = filter_dataframe(Zafira_1_gp_df_filtered)

Auto5_pm_final = filter_dataframe(Auto_5_pm_df_filtered)
Auto4_pm_final = filter_dataframe(Auto_4_pm_df_filtered)
Auto3_pm_final = filter_dataframe(Auto_3_pm_df_filtered)
Auto2_pm_final = filter_dataframe(Auto_2_pm_df_filtered)
Auto1_pm_final = filter_dataframe(Auto_1_pm_df_filtered)
Zafira_2_pm_df_final = filter_dataframe(Zafira_2_pm_df_filtered)
Zafira_1_pm_df_final = filter_dataframe(Zafira_1_pm_df_filtered)


U181_60k_df_final = filter_dataframe(U181_60k_df_filtered)
U181_120k_df_final = filter_dataframe(U181_120k_df_filtered)
U166_df_final = filter_dataframe(U166_df_filtered)
U164_df_final = filter_dataframe(U164_df_filtered)

print('Auto5_gp:', Auto5_gp_final.shape)
print('Auto4_gp:', Auto4_gp_final.shape)
print('Auto5_pm:', Auto5_pm_final.shape)
print('Auto4_pm:', Auto4_pm_final.shape)

Auto5_gp_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_blank_filtered.csv')
Auto4_gp_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PUF_120k_blank_filtered.csv')
Auto3_gp_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto3_PUF_120k_blank_filtered.csv')
Auto2_gp_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto2_PUF_120k_blank_filtered.csv')
Auto1_gp_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto1_PUF_120k_blank_filtered.csv')
Zafira_2_gp_df_final.to_csv('/content/drive/My Drive/unknown_screening_data/Zafira2_PUF_120k_blank_filtered.csv')
Zafira_1_gp_df_final.to_csv('/content/drive/My Drive/unknown_screening_data/Zafira1_PUF_120k_blank_filtered.csv')

Auto5_pm_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PF_120k_blank_filtered.csv')
Auto4_pm_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PF_120k_blank_filtered.csv')
Auto3_pm_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto3_PF_120k_blank_filtered.csv')
Auto2_pm_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto2_PF_120k_blank_filtered.csv')
Auto1_pm_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto1_PF_120k_blank_filtered.csv')
Zafira_2_pm_df_final.to_csv('/content/drive/My Drive/unknown_screening_data/Zafira2_PF_120k_blank_filtered.csv')
Zafira_1_pm_df_final.to_csv('/content/drive/My Drive/unknown_screening_data/Zafira1_PF_120k_blank_filtered.csv')

U181_60k_df_final.to_csv('/content/drive/My Drive/unknown_screening_data/U181_60k_blank_filtered.csv')
U181_120k_df_final.to_csv('/content/drive/My Drive/unknown_screening_data/U181_120k_blank_filtered.csv')
U166_df_final.to_csv('/content/drive/My Drive/unknown_screening_data/U166_blank_filtered.csv')
U164_df_final.to_csv('/content/drive/My Drive/unknown_screening_data/U164_blank_filtered.csv')

Auto5_gp: (456, 23)
Auto4_gp: (514, 23)
Auto5_pm: (184, 23)
Auto4_pm: (175, 23)


In [40]:
Autp5_gp_final_ordered = rearrange_by_area_ratio(Auto5_gp_final)
Autp4_gp_final_ordered = rearrange_by_area_ratio(Auto4_gp_final)
Autp5_pm_final_orderde = rearrange_by_area_ratio(Auto5_pm_final)
Autp4_pm_final_ordered = rearrange_by_area_ratio(Auto4_pm_final)
print(Autp5_gp_final_ordered.head())
print(Autp4_gp_final_ordered.head())
print(Autp5_pm_final_orderde.head())
print(Autp4_pm_final_ordered.head())

                                      Component Name  Retention Time  \
0                                  1-Butene-3-ethoxy           4.802   
1  Benzenepropanoic acid, 3,5-bis(1,1-dimethyleth...          33.695   
2              Carbonic acid, tetradecyl vinyl ester           9.683   
3                                  Decane, 3-methyl-           6.708   
4                 Carbonic acid, decyl undecyl ester          11.236   

   Reference m/z        Area     Height         TIC Formula (mol ion)  \
0      45.033504  8317593860  886789709  3169610421            C6H12O   
1     219.174301  3487699203  201822395  2147150518          C35H62O3   
2      43.054165  2929291121  479368820  2724804750          C17H32O3   
3      43.054165  1872334548  296600545  1007438567            C11H24   
4      43.054165  1854075664  323340139  2458815240          C22H44O3   

      CAS No.     SI    RSI  ...  Selected Column Type  Calculated RI  \
0   1476-08-0  633.0  648.0  ...      StandardNonPolar 